In [201]:
from math import ceil
import time
import re
import ast
import sys
import urllib
import time
import random
import pandas as pd
from bs4 import BeautifulSoup
from random import randint
from selenium.webdriver.firefox.webdriver import FirefoxProfile
from urllib.request import urlopen, Request
from urllib.error import HTTPError
from urllib.error import URLError
import requests
from itertools import cycle
import string
from header_list import user_agent_list
from proxies_list import proxies_list_, proxie_random_pool
from FUNCTIONS import proxie_check, strip_count, gen_url_list
from SCRAPPER_FUNCTIONS import session_creator, zip_prop_count, each_page, links_for_props

In [202]:
ua = user_agent_list
proxies = proxies_list_
prx_pool = proxie_random_pool

In [203]:
# proxie_check(proxies)
len(proxies)

19

In [204]:
zip_codes = pd.read_csv('../Data/main_zips.csv')
zip_list = zip_codes['zip'].tolist()
# zip_list = [str(num) if num >= 10000 else '0'+str(num) for num in zip_list]

In [205]:
# rand_zip_list_ap = random.sample(zip_list, len(zip_list))
# ap_prp_count_list = []

In [206]:
# rand_zip_sld = random.sample(zip_list, len(zip_list))
# sold_prp_list = []
# ezl = []

In [207]:
# ap_prp_count_list, rand_zip_list, proxies = zip_prop_count(rand_zip_list_ap, proxies, ap_prp_count_list, ua)
# sold_prp_list, rand_zip_sld, proxies, ezl = zip_prop_count(rand_zip_sld, proxies, sold_prp_list, ua, ezl)

In [208]:
# sold_prp_list = [str(num) for num in sold_prp_list]
# sold_prp_list = [num.replace(
#     '[<div class="homes summary" data-rf-test-id="homes-description"><span class="showingText">Showing </span>', '') for num in sold_prp_list]
# sold_prp_list = [num.replace(
#     ' Homes<span class="summarySeparator ">•</span></div>]', '') for num in sold_prp_list]
# sold_prp_list = [num.replace('20 of ', '') for num in sold_prp_list]

In [209]:
# sdf = pd.DataFrame({'zip': ezl, 'sld': sold_prp_list})
# apdf = strip_count(ap_prp_count_list)
# apdf.to_csv('../Data/apdf.csv')
# sdf.to_csv('../Data/sdf.csv')

# apdf.index = ap_df['zip']
# sdf.index = sdf['zip']

# sdf['sld'] = (sdf['sld']).astype(int)
# apdf['num_ap_homes'] = (ap_df['num_ap_homes']).astype(int)

# sdf['zip'] = (sdf['zip']).astype(int)
# apdf['zip'] = (ap_df['zip']).astype(int)

# sld_pgs = [ceil(num/20) for num in sdf['sld']]
# ap_pgs = [ceil(num/20) for num in apdf['num_ap_homes']]

# sdf['pages'] = sld_pgs
# apdf['pages'] = ap_pgs

# sdf = sdf.sort_values(by='pages')
# apdf = apdf.sort_values(by='pages')

# sdf_under = sdf[sdf['pages'] <= 18]
# sdf_over = sdf[sdf['pages'] > 18]

In [210]:
# ap_url_list = []
# for num in zip_list:
#     num_pages_count = apdf.loc[apdf['zip'] == num].pages.values[0]
#     for i in range(1, num_pages_count+1):
#         url = 'https://www.redfin.com/zipcode/'+str(num) + \
#             '/filter/property-type=house+condo+townhouse,min-baths=1,status=active+pending+contingent/page-' + \
#             str(i)
#         ap_url_list.append(url)


# ap_url_list = [num.replace(r'/page-1^', '') for num in ap_url_list]
# ap_url_list = [num.replace('contingent0', 'contingent/page-10')
#                for num in ap_url_list]
# ap_url_list = [num.replace('contingent1', 'contingent/page-11')
#                for num in ap_url_list]
# ap_url_list = [num.replace('contingent2', 'contingent/page-12')
#                for num in ap_url_list]
# ap_url_list = [num.replace('contingent3', 'contingent/page-13')
#                for num in ap_url_list]

# main_df = pd.DataFrame(columns=['full_address', 'home_link'])

# ap_url_list = sorted(ap_url_list)
# print(ap_url_list)

In [211]:
# sld_under_url_list = []

# for num in sdf_under['zip']:
#     num_pages_count = sdf_under.loc[sdf_under['zip'] == num].pages.values[0]
#     for i in range(1, num_pages_count+1):
#         url = 'https://www.redfin.com/zipcode/'+str(num) + \
#             '/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-' + \
#             str(i)
#         sld_under_url_list.append(url)


# sld_under_url_list = [num.replace('/page-1', '') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr0', '=sold-1yr/page-10') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr1', '=sold-1yr/page-11') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr2', '=sold-1yr/page-12') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr3', '=sold-1yr/page-13') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr4', '=sold-1yr/page-14') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr5', '=sold-1yr/page-15') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr6', '=sold-1yr/page-16') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr7', '=sold-1yr/page-17') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr8', '=sold-1yr/page-18') for num in sld_under_url_list]

# s_main_df = pd.DataFrame(columns=['full_address', 'home_link'])

# sld_under_url_list = sorted(sld_under_url_list)
# print(len(sld_under_url_list))

In [212]:
# active_url_list, sold_url_list = gen_url_list(zip_list)

In [213]:
# a_main_df = pd.DataFrame(columns=['full_address', 'home_link'])
# s_main_df = pd.DataFrame(columns=['full_address', 'home_link'])

In [ ]:
a = randint(0, 5)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
active_url_list, a_main_df, proxies = links_for_props(
    proxies, active_url_list, a_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
active_url_list, a_main_df, proxies = links_for_props(
    proxies, active_url_list, a_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
active_url_list, a_main_df, proxies = links_for_props(
    proxies, active_url_list, a_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
active_url_list, a_main_df, proxies = links_for_props(
    proxies, active_url_list, a_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
active_url_list, a_main_df, proxies = links_for_props(
    proxies, active_url_list, a_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
active_url_list, a_main_df, proxies = links_for_props(
    proxies, active_url_list, a_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
active_url_list, a_main_df, proxies = links_for_props(
    proxies, active_url_list, a_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
active_url_list, a_main_df, proxies = links_for_props(
    proxies, active_url_list, a_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
active_url_list, a_main_df, proxies = links_for_props(
    proxies, active_url_list, a_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
active_url_list, a_main_df, proxies = links_for_props(
    proxies, active_url_list, a_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
active_url_list, a_main_df, proxies = links_for_props(
    proxies, active_url_list, a_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
active_url_list, a_main_df, proxies = links_for_props(
    proxies, active_url_list, a_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
active_url_list, a_main_df, proxies = links_for_props(
    proxies, active_url_list, a_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
active_url_list, a_main_df, proxies = links_for_props(
    proxies, active_url_list, a_main_df, ua)

2
37.224.84.90:8080
864
https://www.redfin.com/zipcode/90650/filter/sort=lo-price,property-type=house
3.590341329574585
9343.226444482803
Est time remaining: 9343.226444482803 Seconds
Est time remaining: 155.72044074138006 Minutes
Est time remaining: 2.595340679023001 Hours
863
https://www.redfin.com/zipcode/90650/filter/sort=hi-price,property-type=house
0.9901022911071777
12419.675852775574
Est time remaining: 12419.675852775574 Seconds
Est time remaining: 206.99459754625957 Minutes
Est time remaining: 3.449909959104326 Hours
862
https://www.redfin.com/zipcode/90650filter/sort=lo-price,property-type=condo+townhouse
3.080660581588745
17424.196168899536
Est time remaining: 17424.196168899536 Seconds
Est time remaining: 290.4032694816589 Minutes
Est time remaining: 4.840054491360982 Hours
861
https://www.redfin.com/zipcode/90650/filter/sort=hi-price,property-type=condo+townhouse
3.1156973838806152
7146.823034286499
Est time remaining: 7146.823034286499 Seconds
Est time remaining: 119.113

In [199]:
len(active_url_list)

864

In [173]:
a_main_df

,full_address,home_link
0,"11436 Foster Rd, Norwalk, CA 90650",https://www.redfin.com/CA/Norwalk/11436-Foster...
1,"11920 Cheshire St, Norwalk, CA 90650",https://www.redfin.com/CA/Norwalk/11920-Cheshi...
2,"11916 Hermosura St, Norwalk, CA 90650",https://www.redfin.com/CA/Norwalk/11916-Hermos...
3,"10652 Ohm Ave, Norwalk, CA 90650",https://www.redfin.com/CA/Norwalk/10652-Ohm-Av...
4,"11831 Lowemont St, Norwalk, CA 90650",https://www.redfin.com/CA/Norwalk/11831-Lowemo...
5,"11818 Barnwall St, Norwalk, CA 90650",https://www.redfin.com/CA/Norwalk/11818-Barnwa...
6,"13054 Liggett St, Norwalk, CA 90650",https://www.redfin.com/CA/Norwalk/13054-Ligget...
7,"14508 Painter Ave, Norwalk, CA 90650",https://www.redfin.com/CA/Norwalk/14508-Painte...
8,"14423 Wheatstone Ave, Norwalk, CA 90650",https://www.redfin.com/CA/Norwalk/14423-Wheats...
9,"14344 Corby Ave, Norwalk, CA 90650",https://www.redfin.com/CA/Norwalk/14344-Corby-...


In [ ]:
a = randint(0, 5)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
sold_url_list, s_main_df, proxies = links_for_props(
    proxies, sold_url_list, s_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
sold_url_list, s_main_df, proxies = links_for_props(
    proxies, sold_url_list, s_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
sold_url_list, s_main_df, proxies = links_for_props(
    proxies, sold_url_list, s_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
sold_url_list, s_main_df, proxies = links_for_props(
    proxies, sold_url_list, s_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
sold_url_list, s_main_df, proxies = links_for_props(
    proxies, sold_url_list, s_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
sold_url_list, s_main_df, proxies = links_for_props(
    proxies, sold_url_list, s_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
sold_url_list, s_main_df, proxies = links_for_props(
    proxies, sold_url_list, s_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
sold_url_list, s_main_df, proxies = links_for_props(
    proxies, sold_url_list, s_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
sold_url_list, s_main_df, proxies = links_for_props(
    proxies, sold_url_list, s_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
sold_url_list, s_main_df, proxies = links_for_props(
    proxies, sold_url_list, s_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
sold_url_list, s_main_df, proxies = links_for_props(
    proxies, sold_url_list, s_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
sold_url_list, s_main_df, proxies = links_for_props(
    proxies, sold_url_list, s_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
sold_url_list, s_main_df, proxies = links_for_props(
    proxies, sold_url_list, s_main_df, ua)
time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
sold_url_list, s_main_df, proxies = links_for_props(
    proxies, sold_url_list, s_main_df, ua)

In [160]:
sld_under_url_list = sld_under_url_list[1:]
print(len(sld_under_url_list))

In [614]:
sld_under_url_list, s_main_df = links_for_props(
    proxies, sld_under_url_list, s_main_df, ua)

In [616]:
list_ = list(s_main_df.full_address.values)
set([num[-5:] for num in list_])

In [612]:
main_df.to_csv('../Data/main_df_ap.csv')
s_main_df.to_csv('../Data/s_main.csv')

In [21]:
s_main_df = pd.read_csv('../Data/s_main.csv')
main_df = pd.read_csv('../Data/main_df_ap.csv')

In [ ]:
#     all_home_feats = soup.findAll('div', {'class':"amenities-container"})
#     for num in all_home_feats:
#         print(num.findAll('li'))

#     other_home_feats = soup.findAll('span', {'data-rf-test-id':"propertyDetails"})
#     print(f'other_home_feats = {other_home_feats}')
#     other_home_feats_vals = soup.findAll('span', {'class':"content font-weight-roman"})
#     print(f'other_home_feats_vals = {other_home_feats_vals}')
# both are in soup.findAll('div', {'class':"keyDetail font-size-base"})

#     home_full_feat_desc = soup.findAll('span', {'class': 'statsLabel'})
#     for desc in home_full_feat_desc:
#         print(f'hffd = {desc}')
#         homefeat_desc.append(str(desc.contents[0]))

#     home_full_feat_info = soup.findAll('span', {'class': 'statsValue'})
#     for info in home_full_feat_info:
#         print(f'info = {info}')
#         homefeat_info.append(str(info.contents[0]))

#     school_grades = soup.findAll('div', {'class': 'nearby-schools-grades'})
#     for grade in school_grades:
#         print(grade.text)

#     school_assigned = soup.findAll(
#         'span', {'class': 'assigned-label zsg-fineprint'})

#     for assi in school_assigned:
#         print(assi.text)

#     school_dist = soup.findAll('div', {'class': 'nearby-schools-distance'})
#     for dist in school_dist:
#         print(dist.text)

#     school_rating = soup.findAll('div', {'class': 'nearby-schools-rating'})
#     for rating in school_rating:
#         print(rating.span.text)

#     school_name_regex = r'school-name notranslate$'
#     school_name = soup.findAll('a', {'class': 'school-name notranslate'})
#     for mane in school_name:
#         print(mane.next_element)

#     homefeat_desc = []
#     homefeat_info = []

#     new_dict = dict(zip(homefeat_desc, homefeat_info))
#     new_dict['address'] = str(home_address_MLS)

# #     new_dict['home_description'] = str(home_description.attrs['content'])

# lots of info:

#     estimate = soup.find('div', {'data-rf-test-name': 'avmValue"'})
#     print(f'estimate = {estimate}')

#     home_hist_src = soup.findAll('td', {'class': 'zsg-sm-hide'})
#     print(home_hist_src)

In [203]:
def each_property(url, hdr, proxy):
    soup = session_creator(proxy, ua, url)

    price_regex = r'\$\S+\s+'
    bed_regex = r'\d+\S?\d?\d?Bed'
    bath_regex = r'\d+\S?\d?\d?Bath'
    size_regex = r'\d+\S?\d?\d?\s?Sq'
    yr_blt_regex = r'Built: \d+'
    status_regex = r'Status: \w+'
    home_dict = dict()

    home_address_MLS = soup.title.string
    add_list = text.split(sep=' |')
    zip_code = re.search('\d+$', add_list[0])
    mls_num = re.search('\d+$', add_list[1])
    home_add = re.search('(.+?),', home_address_MLS)
    home_city = re.search(',(.+?),', home_address_MLS)
    home_state = add_list[0].replace(home_add.group(1), '').replace(
        home_city.group(1), '').replace(zip_code.group(), '').replace(
        ',', '').replace(' ', '')

    home_dict['address'] = home_add.group(1)
    home_dict['city'] = home_city.group(1)
    home_dict['zip_code'] = zip_code.group()
    home_dict['state'] = home_state
    home_dict['mls_num'] = mls_num.group()

    all_top = soup.findAll('div', {'class': 'HomeInfo inline-block'})
    price_list = []
    for num in all_top:
        a = num.text
        price_list.extend(re.findall(price_regex, a))
        home_dict['num_beds'] = (re.findall(bed_regex, a))
        home_dict['num_baths'] = (re.findall(bath_regex, a))
        home_dict['home_sqft'] = (re.findall(size_regex, a))
        home_dict['yr_blt'] = (re.findall(yr_blt_regex, a))
        home_dict['status'] = (re.findall(status_regex, a))

    home_dict['redfin_est'] = [num for num in price_list if 'Redfin' in num]
    home_dict['sold_price'] = [num for num in price_list if 'Last' in num]

    home_description = soup.find('p', {'class': 'font-b1'})
    home_dict['description'] = home_description.span.text

    all_home_feats = soup.findAll('span', {'class': "entryItemContent"})
    feats = [num.text for num in all_home_feats]

    prop_hist = soup.findAll(
        'div', {'id': 'propertyHistory-expandable-segment'})
    prop_history = [num.text for num in prop_hist]

    school_info = soup.findAll('div', {'class': "name-and-info"})
    schools = [num.text for num in school_info]

    return home_dict, feats, prop_history, schools

In [166]:
header = random.sample(ua, 1)[0]
proxy = random.sample(proxies, 1)[0]
sold_url_list = s_main_df['home_link'].tolist()
url = random.sample(sold_url_list, 1)[0]
print(proxy)

home_dict, feats, prop_history, schools = each_property(url, header, proxy)

104.37.97.15:39471


In [204]:
schools

['Ortega (jose) Elementary SchoolPublic • K to 5 • Serves this homeGreatSchools Rating: 7/10Parent Rating: ',
 'Aptos Middle SchoolPublic • 6 to 8 • Serves this homeGreatSchools Rating: 7/10Parent Rating: ',
 'Leadership High SchoolCharter • 9 to 12 • Serves this homeGreatSchools Rating: 4/10Parent Rating: ',
 'Balboa High SchoolPublic • 9 to 12 • Serves this homeGreatSchools Rating: 7/10Parent Rating: ',
 'Lowell High SchoolPublic • 9 to 12 • Serves this homeGreatSchools Rating: 10/10Parent Rating: ']

In [205]:
prop_history

['DateEvent & SourcePriceAppreciationNov 30, 2017Sold (MLS) (Sold)San Francisco MLS #463469$1,410,000—Oct 19, 2017PendingSan Francisco MLS #463469——Oct 6, 2017Listed (Active)San Francisco MLS #463469$1,295,000—See all property history']

In [206]:
feats

['# of Bedrooms: 3',
 '3 Bedrooms on Main Level',
 '# of Baths: 3',
 '2 Baths on Main Level',
 '1 Bath on Lower Level',
 'Living Room: View',
 'Living Room on Main Level',
 'Family Room on Lower Level',
 'Dining Room on Main Level',
 'Combination Living/Dining Room',
 'Den/Bonus Room: 0',
 '# of Fireplace(s): 1, In Living Room',
 'Laundry Appliances: Washer/Dryer, In Garage',
 'Refrigerator, Freezer, Island, Remodeled',
 'Kitchen on Main Level',
 'Floors: Hardwood',
 '# of Rooms (Total): 0',
 '1 Bath on Lower Level, Family Room on Lower Level',
 '3 Bedrooms on Main Level, 2 Baths on Main Level, Living Room on Main Level, Dining Room on Main Level, Kitchen on Main Level',
 'Heating/Cooling: Central Heating',
 'Parking Type: On Site',
 '# of Garage Spaces: 1',
 '# of Parking Spaces: 1',
 'Garage',
 'Parking Access: Side-by-Side',
 'Deck(s)',
 'Monthly Rent: 0.00',
 'Unconfirmed Co-op Fee: 0.00',
 'Zoning: RH-1',
 'View(s): Ocean',
 'Lot Measurement: Square Feet',
 'Per Architect',
 'Deck